<H3># Donut Model<H3>

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #cuda 11.8
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 #cuda 12.6
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 #cuda 12.1
!pip3 install ipykernel ipython IProgress jupyter ultralytics json numpy transformers torchvision accelerate datasets sentencepiece pillow

  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/7 [tqdm]
   ----------- ---------------------------- 2/7 [regex]
   ----------------- ---------------------- 3/7 [pyyaml]
   ---------------------- ----------------- 4/7 [huggingface-hub]
   ---------------------- ----------------- 4/7 [huggingface-hub]
   ---------------------- ----------------- 4/7 [huggingface-hub]
   ---------------------- ----------------- 4/7 [huggingface-hub]
   ---------------------- ----------------- 4/7 [huggingface-hub]
   ---------------------------- ----------- 5/7 [tokenizers]
   ---------------------------------- ----- 6/7 [transformers

In [ ]:
import torch
import os
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    print("CUDA devices:", torch.cuda.device_count())        
    torch_device = torch.device("cuda:0")
    print("Current CUDA device:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
    


PyTorch version: 2.7.0+cu128
CUDA available: True
CUDA devices: 2
Current CUDA device: 0
CUDA device name: NVIDIA GeForce RTX 4090


In [ ]:
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install --upgrade accelerate
#!pip install accelerate==0.30.0
!pip install tf-keras
!pip install sentencepiece
!pip install protobuf


In [ ]:
#!pip install numpy<2.0.0
print(np.__version__)

2.2.5


In [ ]:
import transformers
print(transformers.__version__)
from transformers import DonutProcessor, VisionEncoderDecoderModel
import tqdm as notebook_tqdm
import numpy as np


c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.52.3


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# !pip install ipykernel
import json

RESULT_DIR = 'results7'
IMAGES_DIR = r'D:\AI_Challenge\DeHaDo-AI\DEHADO-AI_TRAINING_DATASET_PHASE_I\IMAGES_750'
GT_LABELS_DIR = r'D:\AI_Challenge\DeHaDo-AI\DEHADO-AI_TRAINING_DATASET_PHASE_I\LABELS_750'

def read_gt_text(txt_path):
    """
    Read ground truth JSON file and return list of text values.
    """
    result = []
    if not os.path.exists(txt_path):
        return result # Return empty list if file doesn't exist
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            # Assuming `data` is a list where each element is an object containing 'Field value'            
            if isinstance(data, list):
                for item in data:
                    if isinstance(item, dict) and 'Field value' in item:
                        value = item['Field value']
                        if isinstance(value, str):
                            result.append(value.strip())
                        elif isinstance(value, list) and value: # Handle lists of strings
                            result.append(' '.join(map(str, value)).strip()) # Join elements if it's a list
                        else:
                            result.append('') # Append empty string for unexpected formats
                    else:
                        print(f"Warning: Skipping unexpected item format in JSON file: {item}")                        
                        pass
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {txt_path}")
    except Exception as e:
        print(f"Error reading ground truth text from {txt_path}: {e}")

    return result # Return a list of strings

In [ ]:
import os
from transformers import DonutProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import torch
from PIL import Image

# Fine-tune Donut Transformer for Document OCR


# --- Prepare data for Donut ---
# Each sample: {"image": <PIL.Image>, "text": <target string>}

def build_donut_dataset(images_dir, ocr_txt_dir):
    samples = []
    img_list = [] 
    img_list = os.listdir(images_dir)

    for img_file in img_list[:50]:
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        img_path = os.path.join(images_dir, img_file)
        base_filename = os.path.splitext(img_file)[0]
        gt_txt_path = os.path.join(ocr_txt_dir, base_filename + '.json')
        gt_texts = read_gt_text(gt_txt_path)
        # For Donut, we want the full image and all fields as a single string
        try:
            img = Image.open(img_path).convert("RGB")
            target_text = "\n".join(gt_texts)
            samples.append({"image": img, "text": target_text})
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    return samples

donut_samples = build_donut_dataset(IMAGES_DIR, GT_LABELS_DIR)
donut_dataset = Dataset.from_list(donut_samples)

In [ ]:
# Load pre-trained configuration
config = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base").config

# Adjust hyperparameters
config.learning_rate = 5e-5  # Reduce learning rate for fine-tuning
config.batch_size = 8       # Set batch size based on GPU capacity
config.num_train_epochs = 10 # Number of epochs

print(config)

VisionEncoderDecoderConfig {
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "batch_size": 8,
  "decoder": {
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "attention_dropout": 0.0,
    "classifier_dropout": 0.0,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "dropout": 0.1,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layerdrop": 0.0,
    "encoder_layers": 12,
    "init_std": 0.02,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "max_position_embeddings": 1536,
    "model_type": "mbart",
    "num_hidden_layers": 12,
    "scale_embedding": true,
    "torch_dtype": "float32",
    "use_cache": true,
    "vocab_size": 57525
  },
  "encoder": {
    "attention_probs_dropout_prob": 0.0,
    "depths": [
      2,
      2,
      14,
      2
    ],
    "dr

In [ ]:
# --- Load Donut Processor and Model ---
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base",config=config)

# Fix: Set decoder_start_token_id if not set
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = processor.tokenizer.cls_token_id

# Fix: Set pad_token_id if not set
if model.config.pad_token_id is None:
    model.config.pad_token_id = processor.tokenizer.pad_token_id


model.to(torch_device)

# --- Preprocess function ---
def preprocess_donut(example):
    pixel_values = processor(example["image"], return_tensors="pt").pixel_values[0]
    labels = processor.tokenizer(
        example["text"], add_special_tokens=True, max_length=512, padding="max_length", truncation=True, return_tensors="pt"
    ).input_ids[0]
    labels[labels == processor.tokenizer.pad_token_id] = -100  # Ignore padding in loss
    return {"pixel_values": pixel_values, "labels": labels}

donut_dataset = donut_dataset.map(preprocess_donut)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Map: 100%|██████████| 50/50 [01:46<00:00,  2.13s/ examples]


In [ ]:
# --- Split train/test ---
donut_dataset = donut_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = donut_dataset["train"]
eval_dataset = donut_dataset["test"]

In [ ]:
# --- Training Arguments ---
training_args = Seq2SeqTrainingArguments(
    output_dir="./donut-ocr-finetune",
    per_device_train_batch_size=1,  # Reduce from 2 to 1
    per_device_eval_batch_size=1,   # Reduce from 2 to 1
    predict_with_generate=True,
    num_train_epochs=10,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    report_to="none"
)

In [ ]:
from transformers import Seq2SeqTrainer

class DetachLossSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        # Detach loss from computation graph to avoid memory leaks
        loss = outputs.loss.detach()
        return (loss, outputs) if return_outputs else loss

    def training_step(self, model, inputs,hiddens):
        # Standard training step
        loss = super().training_step(model, inputs)
        # Clear CUDA cache after each batch to help avoid OOM
        import torch
        torch.cuda.empty_cache()
        return loss

    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix: str = "eval",
    ):
        # Optionally clear cache after each evaluation batch as well
        output = super().evaluation_loop(
            dataloader,
            description,
            prediction_loss_only=prediction_loss_only,
            ignore_keys=ignore_keys,
            metric_key_prefix=metric_key_prefix,
        )
        import torch
        torch.cuda.empty_cache()
        return output

# --- Trainer ---
def data_collator(batch):
    pixel_values = torch.stack([torch.tensor(x["pixel_values"]) if not isinstance(x["pixel_values"], torch.Tensor) else x["pixel_values"] for x in batch])
    labels = torch.stack([torch.tensor(x["labels"]) if not isinstance(x["labels"], torch.Tensor) else x["labels"] for x in batch])
    return {"pixel_values": pixel_values, "labels": labels}

trainer = DetachLossSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=processor.tokenizer,
    data_collator=data_collator
)

# --- Fine-tune ---
try:
    trainer.train()
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("CUDA OOM error caught. Emptying cache and retrying...")
        torch.cuda.empty_cache()
        # Optionally, you can retry or exit gracefully here
        trainer.train()  # Uncomment to retry (may OOM again if batch size is too large)
    else:
        raise

# Optionally, clear cache after training as well
torch.cuda.empty_cache()



c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\torch\nn\parallel\data_parallel.py:38: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


CUDA OOM error caught. Emptying cache and retrying...


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\torch\nn\parallel\parallel_apply.py", line 97, in _worker
    output = module(*input, **kwargs)
  File "c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\torch\nn\modules\module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\torch\nn\modules\module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
  File "c:\Users\Anand\miniconda3\envs\torch_env2\Lib\site-packages\transformers\models\vision_encoder_decoder\modeling_vision_encoder_decoder.py", line 525, in forward
    encoder_outputs = self.encoder(
        pixel_values=pixel_values,
    ...<3 lines>...
        **kwargs_encoder,
    )
  File "c:\Users\Anand\miniconda3\envs\torch_env2\L

In [ ]:
# Save the fine-tuned model
output_dir = "./fine_tuned_donut"
model.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

# --- Inference Example ---
def donut_ocr_infer(image_path, processor, model):
    img = Image.open(image_path).convert("RGB")
    pixel_values = processor(img, return_tensors="pt").pixel_values
    outputs = model.generate(pixel_values.to(model.device), max_length=512)
    result = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

TEST_IMAGE = r'D:\AI_Challenge\DeHaDo-AI\DEHADO-AI_TRAINING_DATASET\IMAGES_750\MIT_1.jpg'
# Example usage:
result_text = donut_ocr_infer(TEST_IMAGE, processor, model)
print(result_text)

In [ ]:
import torch

# Export to ONNX
dummy_input = torch.rand(1, 3, 256, 256)  # Replace with your input size
onnx_path = "./donut_model.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
)
print(f"Model exported to {onnx_path}")